# Reconnaissance de caractères et de symboles

In [9]:
import numpy as np
import cv2
import os
from pyzbar import pyzbar
from IPython.display import display, Image
import ipywidgets as widgets
import threading

## Fonctions de détection dans une image

In [12]:
def decode_image(img):
    codes = pyzbar.decode(img)
    return codes

In [13]:
def update_image(img, code):
    (x, y, w, h) = code.rect
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)

    codeData = code.data.decode("utf-8")
    codeType = code.type

    text = "{} ({})".format(codeData, codeType)
    cv2.putText(img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    return img

In [17]:
def decode_and_update_image(original):
    if original is None:
        return [], original, original
    
    updated = original.copy()
    img = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

    codes = decode_image(img)
    for code in codes:
        updated = update_image(updated, code)

    return codes, original, updated

In [15]:
def decode_and_update_image_from_path(img_path):
    original = cv2.imread(img_path)

    return decode_and_update_image(original)

## Tests des fonctions

In [44]:
EMPTY_IMAGES_PATH = "./data/images/empty"
BARCODE_IMAGES_PATH = "./data/images/barcode"
QR_IMAGES_PATH = "./data/images/qrcode"

OUTPUT_PATH = "./data/outputs/"

In [45]:
def test_code_detection():
    # boucle sur les dossiers d'images
    for path in [EMPTY_IMAGES_PATH, BARCODE_IMAGES_PATH, QR_IMAGES_PATH]:
        # boucle sur les images du dossier
        for img_path in os.listdir(path):
            full_path = path + "/" + img_path
            print(full_path)

            codes, original, updated = decode_and_update_image_from_path(full_path)

            # print les codes trouvés
            for code in codes:
                codeData = code.data.decode("utf-8")
                codeType = code.type
                print("[INFO] Trouvé : {} code : {}".format(codeType, codeData))

            cv2.imwrite(OUTPUT_PATH + "/" + img_path, updated)

            print("\n")

In [46]:
test_code_detection()

./data/images/empty/img_001.jpg


./data/images/barcode/img_001.png
[INFO] Trouvé : EAN13 code : 0073935942400


./data/images/qrcode/Nestle-and-Google-launch-KitKat-QR-code.jpg
[INFO] Trouvé : QRCODE code : https://goo.gl/9aYrMF




## Fonctions de détection dans une vidéo

In [59]:
video_path = "./data/videos/v1.mp4"
output_path = "./data/outputs/output.mp4"

cap = cv2.VideoCapture(video_path)
width = int(cap.get(3))
height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height), True)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Erreur dans la lecture de la frame")
        break

    codes, original, updated = decode_and_update_image(frame)

    out.write(updated)

cap.release()
out.release()

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Erreur dans la lecture de la frame


## Fonctions de détection à partir d'une webcam

In [27]:
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)



def view(button):
    previous_codes = None
    cap = cv2.VideoCapture(0)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        
        codes, original, updated = decode_and_update_image(frame)

        new_codes = []
        if (previous_codes is None or previous_codes != codes) and len(codes):
            previous_codes = codes
            print(codes)
        
        _, frame = cv2.imencode('.jpeg', updated)
        display_handle.update(Image(data=frame.tobytes()))
        
        if stopButton.value==True:
            cap.release()
            display_handle.update(None)


display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

None

[Decoded(data=b'333831458222', type='CODE128', rect=Rect(left=386, top=210, width=0, height=0), polygon=[Point(x=386, y=210)], quality=1, orientation='UP')]
[Decoded(data=b'333831458222', type='CODE128', rect=Rect(left=181, top=213, width=2, height=26), polygon=[Point(x=181, y=213), Point(x=183, y=239)], quality=2, orientation='UP')]
[Decoded(data=b'333831458222', type='CODE128', rect=Rect(left=184, top=243, width=1, height=10), polygon=[Point(x=184, y=243), Point(x=185, y=253)], quality=2, orientation='UP')]
[Decoded(data=b'333831458222', type='CODE128', rect=Rect(left=184, top=247, width=0, height=0), polygon=[Point(x=184, y=247)], quality=1, orientation='UP')]
[Decoded(data=b'333831458222', type='CODE128', rect=Rect(left=185, top=247, width=0, height=8), polygon=[Point(x=185, y=247), Point(x=185, y=255)], quality=2, orientation='UP')]
[Decoded(data=b'333831458222', type='CODE128', rect=Rect(left=182, top=217, width=2, height=20), polygon=[Point(x=182, y=217), Point(x=184, y=237)], q

Exception in thread Thread-22 (view):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_11596/3372035886.py", line 27, in view
cv2.error: OpenCV(4.8.1) /io/opencv/modules/imgcodecs/src/loadsave.cpp:1113: error: (-215:Assertion failed) !image.empty() in function 'imencode'

